In [1]:
# Установка библиотек
%pip install bs4 openpyxl tqdm \
             multiprocess pandas numpy \
             openpyxl scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# если запускать в колабе, надо еще поменять пути фвйлов
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [418]:
# Импорт библиотек
import re
from datetime import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocess import Pool
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [454]:
rbc_df1 = pd.read_excel('rbc_arts.xlsx')
rbc_df1 = rbc_df1.drop(columns=['Unnamed: 0'])

rbc_df2 = pd.read_excel('rbc_econ_and_russia_arts.xlsx')
rbc_df2 = rbc_df2.drop(columns=['Unnamed: 0'])

rbc_df = pd.concat([rbc_df1, rbc_df2])

In [455]:
# их данных мало и они вносят шум, так как были найдены по тегу, а не в специальном разделе на РБК
rbc_df = rbc_df[~rbc_df['topic'].isin(['Бывший СССР'])] 

In [456]:
rbc_df.rename(columns={'topic_id': 'bloc',
                       'content': 'text'},
               inplace=True)

rbc_df.drop(columns=['datetime'], inplace=True)

In [ ]:
lenta_ru_df1 = pd.read_csv("lenta_arts.csv")
lenta_ru_df2 = pd.read_excel("lenta_arts_russia.xlsx")

# lenta_ru_df = pd.concat([lenta_ru_df1, lenta_ru_df2])
# добавление новых данных по теме Россия из ленты не дало улучшения, поэтому не буду включать их в выборку
lenta_ru_df = pd.concat([lenta_ru_df1])
lenta_ru_df = lenta_ru_df.drop(columns=['docid', 'domain', 'status', 'tags',
                                        'part', 'modified', 'snippet', 'rightcol'])
lenta_ru_df = lenta_ru_df[~lenta_ru_df['text'].isin(['', None])]
lenta_ru_df = lenta_ru_df[rbc_df.columns]



print(lenta_ru_df.shape)

(28736, 5)


## 1. Предобработка данных

In [458]:
def preprocessing(df: pd.DataFrame):
  df_copy = df.copy(deep=True)
  df_copy = df_copy.drop_duplicates() # удаляем дупликаты
  df_copy = df_copy.drop_duplicates(subset=['text'])  # повторы текстов
  df_copy = df_copy.dropna(subset=['text']) # пустые тексты
  df_copy = df_copy[~(df_copy['text'].str.split().str.len() < 20)] # тексты из менее, чем 50 слов
  df_copy = df_copy.reset_index(drop=True)
  return df_copy

In [459]:
print(rbc_df.shape, preprocessing(rbc_df).shape)
rbc_df = preprocessing(rbc_df)

(12852, 5) (12046, 5)


In [460]:
print(lenta_ru_df.shape, preprocessing(lenta_ru_df).shape)
lenta_ru_df = preprocessing(lenta_ru_df)

(28736, 5) (26881, 5)


In [461]:
# уберем лишние приписки, которые не несут смысла, а являются лишь справочной информацией
# у рбк таких проблем нет
@np.vectorize
def remove_metadata(text: str) -> str:
  '''
    Deletes metadata such as "/ photo from Chunga changa /", and "Фото: {word} {word}".
  '''
  patterns = [
              re.compile(r'(?:\w+\s*){,2}\s*/\s*(?:\w+\s*)/?', flags=re.IGNORECASE),
              re.compile(r'Фото:(\s*(?:[\w.-]*\/?)){2}', flags=re.IGNORECASE),
              ]
  new_text = text
  for pattern in patterns:
    new_text = re.sub(pattern, ' ', new_text)
  return new_text.strip()


# with Pool(5) as pool:
#   lenta_ru_df['text'] = pool.map(remove_metadata, lenta_ru_df['text'])
# lenta_ru_df['text'] = lenta_ru_df['text'].apply(str)


В ходе экспериментов выяснилось, что при удалениии этих данных, качество модели падает, поэтому этот этап придется пропустить.

In [462]:
from string import punctuation
# удаление пунктуации из текста
def clean_from_punct(text: str) -> str:
    return re.sub(rf'[{punctuation}]', ' ', text)

Удаление пунктуации также не приводит к улучшению качества модели, тоже пропускаем.

In [463]:
# в названиях статей тоже содержится важная информация, которая поможет идентифицировать тему, 
# поэтому добавим их к основным текстам 
arts_df = pd.concat([lenta_ru_df, rbc_df])
arts_df.loc[:, 'text'] = arts_df['title'] + arts_df['text']
# arts_df.loc[:, 'text'] = arts_df['text'].apply(clean_from_punct)

In [464]:
arts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38927 entries, 0 to 12045
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   topic   38927 non-null  object
 1   url     38927 non-null  object
 2   title   38927 non-null  object
 3   text    38927 non-null  object
 4   bloc    38927 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


Найдем соответствие между кодом блока, его названием и кодом в соревновании:

* 1 - Россия - 0
* 37 - Силовые структуры - 2
* 3 - Бывший СССР - 3
* 4 - Экономика - 1
* 5 - Наука и техника - 8
* 8 - Спорт - 4
* 48 - Туризм - 7
* 87 - Здоровье - 5

In [465]:
topic_name_to_lenta_id = {
    'Россия': 1,
    'Силовые структуры': 37,
    'Бывший СССР': 3,
    'Экономика': 4,
    'Наука и техника': 5,
    'Спорт': 8,
    'Туризм': 48,
    'Здоровье': 87,
    'Строительство': 6,
}

In [466]:
arts_df_new = arts_df[arts_df.bloc.isin([1, 37, 3, 4, 5, 8, 48, 87, 6])]

TagsMap = {1 : 0, 3 : 3, 4 : 1, 5 : 8, 8 : 4, 37 : 2, 48 : 7, 87 : 5,
           6 : 6}


arts_df_new.loc[:, 'topic'] = arts_df_new['bloc'].map(TagsMap)

In [467]:
arts_df_new.loc[:, 'topic'].unique()

array([0, 1, 2, 3, 4, 5, 7, 8, 6], dtype=object)

In [468]:
pd.concat([arts_df_new.value_counts('topic'),
           arts_df_new.value_counts('topic', normalize=True).round(3)], axis=1)

,count,proportion
topic,,
0,6902,0.177
1,6594,0.169
7,5304,0.136
4,4340,0.111
8,3540,0.091
2,3522,0.090
3,3520,0.090
5,3497,0.090
6,1708,0.044


## 2. Машинное обучение

Загружаем данные и обучаем модель на разбиении трейн-тест

In [469]:
X = arts_df_new[['text']]
y = arts_df_new['topic'].astype(int)

X.shape, y.shape

((38927, 1), (38927,))

In [470]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape

((29195, 1), (9732, 1))

### Baseline-решение

In [471]:
baseline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('scaler', MaxAbsScaler()),
    ('model', LogisticRegression(max_iter=200, random_state=42))
])

baseline.fit(X_train['text'], y_train)
pred = baseline.predict(X_test['text'])

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1723
           1       0.92      0.93      0.92      1654
           2       0.98      0.95      0.96       880
           3       0.92      0.94      0.93       877
           4       1.00      0.99      0.99      1087
           5       0.98      0.99      0.99       838
           6       1.00      0.95      0.97       431
           7       0.96      0.95      0.95      1353
           8       0.98      0.95      0.97       889

    accuracy                           0.95      9732
   macro avg       0.96      0.95      0.95      9732
weighted avg       0.95      0.95      0.95      9732



Для векторизации текста в качестве бейзлайна случае я использую CountVectorizer, то есть метод Bag of words. Это значит, что находятся уникальные слова (токены), а потом для каждого наблюдения (текста) вычисляется частота этого токена. Поэтому кол-во столбцов = кол-ву уникальных токенов.

In [472]:
pipe = Pipeline([
    # заглушки
    ('vectorizer', CountVectorizer()),
    ('scaler', MaxAbsScaler()),
    ('model', LogisticRegression(max_iter=200, random_state=42))
])


param_grid = [
    {
        'model': [LogisticRegression(max_iter=200,
                                     random_state=42)],

        'vectorizer': [CountVectorizer(),
                       TfidfVectorizer()],

        'vectorizer__ngram_range': [(1, 1)],
    },

    {
        'model': [KNeighborsClassifier()],

        'vectorizer': [CountVectorizer(),
                        TfidfVectorizer()],

        'vectorizer__ngram_range': [(1, 1)],
        'model__n_neighbors': [3, 4, 5],
    }
]

clf = GridSearchCV(pipe, param_grid, cv=5, n_jobs=5)

Далее я добавляю метод TF-IDF (term frequency - inverse document frequency). Это отношение частоты вхождения токена в документ и логарифма обратной частоты документов, в которых встречается токен.

В качестве дополнительной модели классификации был применен метод k ближайших соседей.

In [473]:
clf.fit(X_train['text'], y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","[{'model': [LogisticRegre...ndom_state=42)], 'vectorizer': [CountVectorizer(), TfidfVectorizer()], 'vectorizer__ngram_range': [(1, ...)]}, {'model': [KNeighborsClassifier()], 'model__n_neighbors': [3, 4, ...], 'vectorizer': [CountVectorizer(), TfidfVectorizer()], 'vectorizer__ngram_range': [(1, ...)]}]"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",5
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default 

In [475]:
pred = clf.predict(X_test['text']) 

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90      1723
           1       0.94      0.94      0.94      1654
           2       0.98      0.96      0.97       880
           3       0.93      0.94      0.94       877
           4       1.00      0.99      0.99      1087
           5       0.98      0.99      0.99       838
           6       0.99      0.97      0.98       431
           7       0.97      0.96      0.97      1353
           8       0.97      0.96      0.97       889

    accuracy                           0.95      9732
   macro avg       0.96      0.96      0.96      9732
weighted avg       0.95      0.95      0.95      9732



```
Макросредние по метрикам recall и f1-score стали выше на 1%, небольшое улучшение относительно бейзлайна. 
Это значит, что при равном влиянии всех классов, в среднем модель лучше их распознает. 
Взвешенные средние не изменились, то есть с учетом дисбаланса классов модель не стала лучше.
Но дисбаланс классов был добавлен намеренно, чтобы улучшить качество для классов, которые распознавались хуже всего.
Нам равноценно важны все классы, поэтому макросредние тут будут более релевантны.
```

Загружаем тестовые данные, обучаем итоговую модель и делаем прогноз.

In [476]:
Test = pd.read_csv("test_news.csv")
# with Pool(5) as pool:
#   Test['content'] = pool.map(remove_metadata, Test['content'])
# Test['content'] = Test['content'].apply(str).apply(clean_from_punct)

Test.head()

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...


In [477]:
pred = clf.predict(Test['content'])
subm = pd.read_csv("base_submission_news.csv")
subm['topic'] = pred

subm.to_csv("bow_logreg.csv", index=False)

Лучший вариант - без удаления метаданных, пунктуации и с TF-IDF векторизацией.

Почему удаление метаданных и пунктуации не улучшило качество модели - в пунктуации, возможно, при удалении пукнтуации мы также снимаем часть смысла - речевые обороты, перечисления, конец предложения. 

А такие метаданные, как имя автора фото со статьи, как ни странно, тоже может быть связано с определенной тематикой. Например, некоторые фотографы делают только снимки, которые можно отнести к теме "Спорт", поэтому их имя также может быть индикатором определенной темы.

Почему TF-IDF? У нас есть дисбаланс классов, поэтому его применение тут лучше. 
TF-IDF занижает значимость частых и завышает значимость редких токенов. 